<a href="https://colab.research.google.com/github/mdrca23/amz/blob/main/Copy_of_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_to_file = '/content/drive/MyDrive/Colab Notebooks/amaztxtnew.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')
print(text[:500])

Length of text: 230390 characters
Works well. Bass is somewhat lacking but is present. Overall pleased with the item.
The quality on these speakers is insanely good and doesn?t sound muddy when adjusting bass. Very happy with these.
Wish I could give this product more than five stars. Lifesaver.
works great
Great sound and compact. Battery life seems good. Happy with this product.
It works well~~~
Alll good
Love clock radio & CD player.  Easy to operate.
Breaks very easily, and takes a while to load music
Excellent gain


### Read the data

First, look in the text:

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

88 unique characters


### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[61, 62, 63, 64, 65, 66, 67], [84, 85, 86]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(230390,), dtype=int64, numpy=array([54, 75, 78, ...,  1,  2,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

W
o
r
k
s
 
w
e
l
l


In [ ]:
seq_length = 300


The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'W' b'o' b'r' b'k' b's' b' ' b'w' b'e' b'l' b'l' b'.' b' ' b'B' b'a'
 b's' b's' b' ' b'i' b's' b' ' b's' b'o' b'm' b'e' b'w' b'h' b'a' b't'
 b' ' b'l' b'a' b'c' b'k' b'i' b'n' b'g' b' ' b'b' b'u' b't' b' ' b'i'
 b's' b' ' b'p' b'r' b'e' b's' b'e' b'n' b't' b'.' b' ' b'O' b'v' b'e'
 b'r' b'a' b'l' b'l' b' ' b'p' b'l' b'e' b'a' b's' b'e' b'd' b' ' b'w'
 b'i' b't' b'h' b' ' b't' b'h' b'e' b' ' b'i' b't' b'e' b'm' b'.' b'\r'
 b'\n' b'T' b'h' b'e' b' ' b'q' b'u' b'a' b'l' b'i' b't' b'y' b' ' b'o'
 b'n' b' ' b't' b'h' b'e' b's' b'e' b' ' b's' b'p' b'e' b'a' b'k' b'e'
 b'r' b's' b' ' b'i' b's' b' ' b'i' b'n' b's' b'a' b'n' b'e' b'l' b'y'
 b' ' b'g' b'o' b'o' b'd' b' ' b'a' b'n' b'd' b' ' b'd' b'o' b'e' b's'
 b'n' b'?' b't' b' ' b's' b'o' b'u' b'n' b'd' b' ' b'm' b'u' b'd' b'd'
 b'y' b' ' b'w' b'h' b'e' b'n' b' ' b'a' b'd' b'j' b'u' b's' b't' b'i'
 b'n' b'g' b' ' b'b' b'a' b's' b's' b'.' b' ' b'V' b'e' b'r' b'y' b' '
 b'h' b'a' b'p' b'p' b'y' b' ' b'w' b'i' b't' b'h' b' ' b't' b'h

It's easier to see what this is doing if you join the tokens back into strings:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Works well. Bass is somewhat lacking but is present. Overall pleased with the item.\r\nThe quality on these speakers is insanely good and doesn?t sound muddy when adjusting bass. Very happy with these.\r\nWish I could give this product more than five stars. Lifesaver.\r\nworks great\r\nGreat sound and compac'
b't. Battery life seems good. Happy with this product.\r\nIt works well~~~\r\nAlll good\r\nLove clock radio & CD player.  Easy to operate.\r\nBreaks very easily, and takes a while to load music\r\nExcellent gain in radio frequency reception over the stock antenna that came with the radio.\r\neverything I expected '
b'for a great price\r\nLove this small speaker with loud volume, great for the beach\r\nworks as advertised.... nothing else to really say about it\r\nvery good especially for notebooks with micro hdmi and you want to connect it to a computer screen\r\nIt does not copy CDR s to USB as advertised. My CD?s were '
b'recorded on a Sony machine and properly finalized. They

For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Works well. Bass is somewhat lacking but is present. Overall pleased with the item.\r\nThe quality on these speakers is insanely good and doesn?t sound muddy when adjusting bass. Very happy with these.\r\nWish I could give this product more than five stars. Lifesaver.\r\nworks great\r\nGreat sound and compa'
Target: b'orks well. Bass is somewhat lacking but is present. Overall pleased with the item.\r\nThe quality on these speakers is insanely good and doesn?t sound muddy when adjusting bass. Very happy with these.\r\nWish I could give this product more than five stars. Lifesaver.\r\nworks great\r\nGreat sound and compac'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 300), dtype=tf.int64, name=None), TensorSpec(shape=(64, 300), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 300, 89) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  22784     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  91225     
                                                                 
Total params: 4,052,313
Trainable params: 4,052,313
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

array([47,  6,  3, 22, 77, 46, 41, 68,  4, 84, 63, 44, 12, 26, 15, 81, 58,
       53, 45, 45, 51, 22, 36,  9, 30, 61, 28, 46, 63, 22, 38, 75, 18, 24,
        6, 75, 25, 11, 20,  2, 54, 16, 51, 66, 51, 82, 13, 65, 75, 54, 78,
       83, 51, 17, 24, 67, 10, 88, 52, 21, 37, 50, 43, 67, 42, 63, 51, 73,
       24, 50, 47, 20, 80, 82, 86, 25,  3, 12, 50, 63, 32, 23, 51, 67, 69,
       47, 45, 20, 87, 56, 65, 16, 27,  7, 70,  1, 66, 17, 27, 20,  4, 62,
       55,  0, 83, 81, 42, 54, 30, 83, 26, 61, 34,  5, 83,  9, 12, 33, 31,
       31, 27, 46, 28, 20, 37, 36, 75, 87, 66, 56, 57, 33, 84, 24, 76, 50,
       20, 34, 60, 24, 61, 64, 52, 33,  8, 27, 11, 72, 71, 86, 42, 71, 82,
       86, 28, 69, 57, 71, 79, 18,  1, 66, 62, 58, 49, 77, 43, 72, 52, 35,
       50, 18, 87,  3, 25, 12, 27, 87, 20, 57, 80, 28, 21, 47, 28,  4, 63,
       57, 86, 27, 42, 66, 22, 11, 15, 53,  7, 80,  2, 11, 88,  9, 40, 27,
       34,  2, 47, 66, 67, 69, 67, 21, 26, 48,  0, 87, 10, 42, 84, 51, 13,
       21, 65, 14, 84, 40

Decode these to see the text predicted by this untrained model:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ermal pads. Although we are at a state in tech where even at 100C these PN junctions still operate happily for a while. But for longterm durability, the heat created is never good. I can safely say that the charger will last a good year or two, but with that amount of heat produced, you can question'

Next Char Predictions:
 b'P$ 6qOJh!xcM+:/u[VNNT6E(?a=Oc6Go28$o9*4\rW0TfTv,eoWrwT18g)~U5FSLgKcTm8SP4tvz9 +ScA7TgiPN4|Ye0;%j\nf1;4!bX[UNK]wuKW?w:aC#w(+B@@;O=4FEo|fYZBx8pS4C_8adUB&;*lkzKkvz=iZks2\nfb[RqLlUDS2| 9+;|4Zt=5P=!cZz;Kf6*/V%t\r*~(I;C\rPfgig5:Q[UNK]|)KxT,5e.xI>9gu4(nj,RlD8*b2gYv\r~($x.84l5qxiVnl6Yj>=#D2A!(0L?Q!Gz1i*Dh*.HUxD\rNNhGKThG|'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 300, 89)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.4887567, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
tf.exp(example_batch_mean_loss).numpy()

89.010704

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS = 100

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
11/11 [==============================] - 2s 160ms/step - loss: 0.1784
Epoch 2/100
11/11 [==============================] - 2s 156ms/step - loss: 0.1647
Epoch 3/100
11/11 [==============================] - 2s 155ms/step - loss: 0.1516
Epoch 4/100
11/11 [==============================] - 2s 157ms/step - loss: 0.1411
Epoch 5/100
11/11 [==============================] - 2s 157ms/step - loss: 0.1299
Epoch 6/100
11/11 [==============================] - 2s 159ms/step - loss: 0.1187
Epoch 7/100
11/11 [==============================] - 2s 156ms/step - loss: 0.1097
Epoch 8/100
11/11 [==============================] - 2s 156ms/step - loss: 0.1013
Epoch 9/100
11/11 [==============================] - 2s 158ms/step - loss: 0.0944
Epoch 10/100
11/11 [==============================] - 2s 156ms/step - loss: 0.0881
Epoch 11/100
11/11 [==============================] - 2s 159ms/step - loss: 0.0821
Epoch 12/100
11/11 [==============================] - 2s 158ms/step - loss: 0.0770
Epoch 13/100


## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['The'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*200)
print('\nRun time:', end - start)

The original battery lasted over three mextem and that I found to fit to my bigur batteries they stirld over one and day it looks and pintosite worked as intended but the headphones battery died at exactly 2 hours. The description says 6+ hours listening tipe on the stee. I find my easy installation.
Much easier to hook up TV on the patio. Picked up several digital cunce. So I have a couple to perform de, in less than 2 months. Opening the unit reveals the problem....the Headphone Jack secured to the motherboard with two dabs of hot glud I got these and it works great.
Love this radio. It alerts the whole family when any form of weather changes. Plus you can have one in each room.
Fits my tv perfectly
Enjoy crystal clear clarity sound with this product!  Super great quality! Highly recommended! Would make a perfect size power.
Perfect fit. efficinned.
The amplifier made almost no difference in the reception.  I live 1015 miles from the tower and most channels came in fine.  I probab 



The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['The', 'When', 'If', 'Can', 'Why'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*200)
print('\nRun time:', end - start)

tf.Tensor(
[b'Thes was easy and was packed great until they are have 10 mines are comfortable and will last longer, but I think the sound of the monopricer headphones works well. My husband likes it so much that I don?t get to use it often.\r\nThe main reason I picked this clock was because of the time projection. I was leery about it because I didn?t think I would be happy. But, I?m glad I went with this one. My deciding factor was easy to work. Charges all out of my ear Bass gest the KPe 805 smill apain. I a song minutes I found these to be a bit boomy, they almost have nothing but bass. I did however get them to sound great with the use of the PowerAmp Apple TV back on but no picture flickering screen, so tried with an HDMI and DVI connection on similar monitor and that worked. So for the money and the fact it does work I got a little bit that or its needed to plug the usb power cord into. Minor,I had spare and appearsed with the sound and the fact that the secsnonece of 9chargs. Th

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['The'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

The original battery lasted over three years... this one lasted 8 months. The &#34;Designed for long li


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

11/11 [==============================] - 4s 144ms/step - loss: 4.6056


Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 30

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.0111

Epoch 1 Loss: 3.5682
Time taken for 1 epoch 2.43 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 3.1473

Epoch 2 Loss: 3.0738
Time taken for 1 epoch 1.78 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 2.9989

Epoch 3 Loss: 2.9274
Time taken for 1 epoch 1.78 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 2.8507

Epoch 4 Loss: 2.7647
Time taken for 1 epoch 1.78 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 2.6649

Epoch 5 Loss: 2.6261
Time taken for 1 epoch 1.92 sec
________________________________________________________________________________
Epoch 6 Batch 0 Loss 2.5297

Epoch 6 Loss: 2.5319
Time taken for 1 epoch 1.78 sec
________________________________________________________________________________
Epoch 7 Batch 0 Loss 2